# Théorie de l'information

C'est un champ d'études concernant la quantification, le stockage et la communication d'informations. Les branches principales de cette théorie sont : 
le codage de l'information ;
la mesure quantitative de redondance d'un texte ;
la compression de données ;
la cryptographie.

### théorie des codes 
Tout ce qui touche à la transmission de données binaires (voire trinaires ou q-aires), un code est caractérisé par une certaine quantité de bits d'informations et une quantité de bits de controle, pour pouvoir trouver et corriger les erreurs. Un code à donc une distance de Hamming minimale (nombre de bits differents entre 2 mots codés, par exemple entre 00110 et 10001 la distance est de 4) et un rendement (nombre de bits d'information/nombre de bits total).

Prenons un exemple : on décide de coder des blocs de 2 bits ainsi -> on écrit les 2 bits, puis leur inverse puis 0 si les 2 bits de bases sont pair ou 1 sinon
Donc 4 possibilitées :

| bits d'information | code   |
|------|------|
|   00  | 00110|
|   01  | 01101|
|   10  | 10011|
|   11  | 11000|

La distance de hamming minimale est 3, on peut le voir simplement avec ce tableau :

|codes | 00110| 01101|10011|11000|
|------|------|------|------|------|
|00110 |   0  | 3| 3  | 4|
|01101 |   3  | 0| 4  | 3|
|10011 |   3  | 4| 0  | 3|
|11000 |   4  | 3| 3  | 0|

Le rendement est de 2/5.
Imaginons maintenant que l'on reçoit le message R = 10001, on est certain qu'il y a au moins une erreur car ce n'est pas un des 4 mots possibles. 

### codes linéaires
Un code est linéaire si l'addition de ses informations puis son codage est équivalent à l'addition des informations codées. Prenons le code plus haut pour l'exemple : si on additionne 00 et 01 on obtient 01, une fois codé on a donc le message 01101.
Par contre si on prend le code de 00 -> 00110 et de 01 -> 01101 l'addition ne donne pas 01101. Donc ce n'est pas un code linéaire. Ici notre exemple est loin d'être une méthode optimale, il existe des codes dit "parfait" comme le code de Hamming, ils n'ont donc aucune redondance inutile c'est à dire que l'on ne peut pas faire de meilleur rendement. (voir Annexe 2 document jupyter sur le code de Hamming ou la partie correspondante dans le rapport).

### codes cycliques
Les codes cycliques sont un sous-ensemble des codes linéaires qui ont la particularité de pouvoir détecter les erreurs, mais aussi de les corriger.

### codes BCH
Un sous-ensemble des codes cycliques qui utilise des polynomes dans un champ défini (le champ de Galois)
L'une des utilitées principales de ces codes est que durant la création du code, on a un contrôle précis du nombres d'erreurs corrigibles par le code. En particulier, il est possible des concevoir des codes BCH binaires qui peuvent corriger plusieurs bits successifs erronés. Un autre avantage des codes BCH est la facilité avec laquelle ils peuvent être décodés grâce à une méthode d'algèbre connue sous le nom de "syndrome decoding". Cela simplifie la conception du décodeur en utilisant de petits composants électroniques.

L'intéret des codes BCH par rapport au code de Hamming par exemple, est qu'une telle approche ne permet que la résolution d'une unique erreur dans un mot. Cette limitation n'est pas compatible avec les CDs qui peuvent subir des rayures ou être localement recouvert d'une poussière ce qui provoquerait plusieurs erreurs successives. 

Les codes BCH sont notamment utilisés dans des applications comme des communications satellites, DVDs, disques durs, la cryptographie post-quantique ou encore les codes bares.

### code de Reed-Solomon
C'est un sous ensemble basé principalement sur les codes BCH, ce type de code correcteur est notamment utilisé dans les CDs.
Voir le document jupyter associé pour plus d'informations.

### ci-dessous une implémentation à l'aide d'une bibliothèque d'un code BCH

In [6]:
from platform import python_version

print(python_version())
# Il faut une version supérieure à 3.5

3.7.3


In [4]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install bchlib

  Using cached https://files.pythonhosted.org/packages/12/39/559d2a0d3351a9d14695c8176057d43e90fcd2bc99be76582a3de4947733/bchlib-0.14.0.tar.gz
  Running setup.py install for bchlib ... error
    Complete output from command /opt/sagemath-9.1/local/bin/python3.exe -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-6qk49e9q/bchlib/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /tmp/pip-record-j1epb95e/install-record.txt --single-version-externally-managed --compile:
    running install
    running build
    running build_ext
    building 'bchlib' extension
    creating build
    creating build/temp.cygwin-3.0.7-x86_64-3.7
    creating build/temp.cygwin-3.0.7-x86_64-3.7/src
    gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wno-unused -I/opt/sagemath-9.1/local/include/python3.7m -c src/bchlib.c -o build/temp.cygwin-3.0.7-x86_64-3.7/src/bchlib.o -std

### Si l'installation échoue sous Jupyter vous pouvez essayer le code en local avec le fichier bch.py

In [ ]:
import bchlib
import hashlib
import os
import random

# on créer un objet BCH
BCH_POLYNOME = 8219
BCH_BITS = 16
bch = bchlib.BCH(BCH_POLYNOME, BCH_BITS)

# données aléatoires
data = bytearray(os.urandom(512))

# encodage dans un packet
ecc = bch.encode(data)
packet = data + ecc

# affichage du hash du packet
sha1_initial = hashlib.sha1(packet)
print('sha1: %s' % (sha1_initial.hexdigest(),))

def bitflip(packet):
    byte_num = random.randint(0, len(packet) - 1)
    bit_num = random.randint(0, 7)
    packet[byte_num] ^= (1 << bit_num)

# make BCH_BITS errors
for _ in range(BCH_BITS):
    bitflip(packet)

# print hash of packet
sha1_corrupt = hashlib.sha1(packet)
print('sha1: %s' % (sha1_corrupt.hexdigest(),))

# de-packetize
data, ecc = packet[:-bch.ecc_bytes], packet[-bch.ecc_bytes:]

# correct
bitflips = bch.decode_inplace(data, ecc)
print('bitflips: %d' % (bitflips))

# packetize
packet = data + ecc

# print hash of packet
sha1_corrected = hashlib.sha1(packet)
print('sha1: %s' % (sha1_corrected.hexdigest(),))

if sha1_initial.digest() == sha1_corrected.digest():
    print('Corrected!')
else:
    print('Failed')

### Sources :
codes correcteurs linéaire https://youtu.be/dYrH5Cbcn6M

Explications simple sur les codes correcteurs d'erreurs : https://youtu.be/q-3BctoUpHE

Cours de Pierre Abbrugiati : http://www.lirmm.fr/~chaumont/download/cours/codescorrecteur/Cours_Pierre_Abbrugiati.pdf

Cours ENS : http://perso.eleves.ens-rennes.fr/people/Emily.Clement/documents/COCO.pdf

Wikipedia : 

https://fr.wikipedia.org/wiki/Th%C3%A9orie_de_l%27information

https://fr.wikipedia.org/wiki/Code_correcteur

https://fr.wikipedia.org/wiki/Code_lin%C3%A9aire

https://fr.wikipedia.org/wiki/Code_cyclique

https://fr.wikipedia.org/wiki/Code_de_Reed-Solomon